# Inference

## Instalation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## Global Variables

In [2]:
dataset_name = (
    input(
        "Enter the name of the generated dataset."
    )
)
print(f"{dataset_name=}")
model_name = (
    input(
        "Enter the name of fine-tuned LLM."
    )
)
print(f"{model_name=}")

Enter the name of the generated dataset.Kacper098/summarization_task
dataset_name='Kacper098/summarization_task'
Enter the name of fine-tuned LLM.Kacper098/Meta-Llama-3.1-8B-Instruct-Assistant-Summarization
model_name='Kacper098/Meta-Llama-3.1-8B-Instruct-Assistant-Summarization'


## Load Fine-tuned LLM

In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8192,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

## Prepare Input Samples

In [4]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
{}

### Response:
{}"""

In [5]:
dataset = load_dataset(dataset_name, split="test")

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/748 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/94 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/94 [00:00<?, ? examples/s]

In [6]:
dataset[0]["instruction"][:1000]

"[![Hugging Face's logo](/front/assets/huggingface_logo-noborder.svg) Hugging Face](/)\n\n  * [ Models](/models)\n  * [ Datasets](/datasets)\n  * [ Spaces](/spaces)\n  * [ Posts](/posts)\n  * [ Docs](/docs)\n  * [ Enterprise](/enterprise)\n  * [Pricing](/pricing)\n  * [Log In](/login)\n  * [Sign Up](/join)\n\n\n\nAmazon SageMaker documentation\n\nRun training on Amazon SageMaker\n\n# Amazon SageMaker\n\n🏡 View all docsAWS Trainium & InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabelEvaluateGradioHubHub Python LibraryHugging Face Generative AI Services (HUGS)Huggingface.jsInference API (serverless)Inference Endpoints (dedicated)LeaderboardsLightevalOptimumPEFTSafetensorsSentence TransformersTRLTasksText Embeddings InferenceText Generation InferenceTokenizersTransformersTransformers.jssmolagentstimm\n\nSearch documentation\n\n`Ctrl+K`\n\nmain EN [ 355](https://github.com/huggingface/hub-docs)\n\n[Hugging Face on Am

In [7]:
dataset[0]["answer"][:1000]

'```markdown\nTL;DR: This guide explains how to train 🤗 Transformers models on AWS SageMaker, managing outputs, using hyperparameters, and enabling checkpointing for efficient deployment.\n```'

## Inference

In [8]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


def generate_text(
    instruction, streaming: bool = True, trim_input_message: bool = False
):
    message = alpaca_prompt.format(
        instruction,
        "",  # output - leave this blank for generation!
    )
    inputs = tokenizer([message], return_tensors="pt").to("cuda")

    if streaming:
        return model.generate(
            **inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True
        )
    else:
        output_tokens = model.generate(**inputs, max_new_tokens=256, use_cache=True)
        output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

        if trim_input_message:
            return output[len(message) :]
        else:
            return output

In [9]:
_ = generate_text(dataset[0]["instruction"], streaming=True)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
[![Hugging Face's logo](/front/assets/huggingface_logo-noborder.svg) Hugging Face](/)

  * [ Models](/models)
  * [ Datasets](/datasets)
  * [ Spaces](/spaces)
  * [ Posts](/posts)
  * [ Docs](/docs)
  * [ Enterprise](/enterprise)
  * [Pricing](/pricing)
  * [Log In](/login)
  * [Sign Up](/join)



Amazon SageMaker documentation

Run training on Amazon SageMaker

# Amazon SageMaker

🏡 View all docsAWS Trainium & InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabelEvaluateGr

In [10]:
generate_text(dataset[0]["instruction"], streaming=False)

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights\n\n### Input:\n[![Hugging Face\'s logo](/front/assets/huggingface_logo-noborder.svg) Hugging Face](/)\n\n  * [ Models](/models)\n  * [ Datasets](/datasets)\n  * [ Spaces](/spaces)\n  * [ Posts](/posts)\n  * [ Docs](/docs)\n  * [ Enterprise](/enterprise)\n  * [Pricing](/pricing)\n  * [Log In](/login)\n  * [Sign Up](/join)\n\n\n\nAmazon SageMaker documentation\n\nRun training on Amazon SageMaker\n\n# Amazon SageMaker\n\n🏡 View all docsAWS Trainium & InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabe